## <b><span style='color:#FF6F61'>|</span> Introduction</b> 

## <b><span style='color:#FF6F61'>|</span> Environment Setup</b> 

## <b><span style='color:#FF6F61'>|</span> Exploratory Data Analysis</b> 

## <b><span style='color:#FF6F61'>|</span> Data Preprocessing</b> 

## <b><span style='color:#FF6F61'>|</span> Modeling</b> 

## <b><span style='color:#FF6F61'>|</span> Hyperparameter Tuning</b> 

## <b><span style='color:#FF6F61'>|</span> Final Model and Conclusion</b> 